# source_code_1.ipynb

This Jupyter notebook produces most of the figures for:  
Verduzco-Flores S., De Schutter E. (2021) "Adaptive plasticity in the spinal cord can produce reaching from scratch and reproduces motor cortex directional tuning."

The figures are produced by executing all the cells below sequentially. Many figures not in the paper are also produced.

**Before executing** make sure you adjust the cell below so it contains the root folder of your Draculab installation. This source code file, and `v3ft3p2ph2_net.py` should be in that folder.

Also make sure to adjust values in the third cell in order to try different configurations for the simulation.

Creation of the Draculab network uses the `v3ft3p2ph2_net.py` file, which contains all parameter dictionaries. 
Naming conventions in this file a bit different from the paper. The $S_A$ population is called `SF`, $S_{PA}$ is called `SPF`, $\alpha$ is called `AL`, $CHG$ is called `SP_CHG`.
Parameter names may also differ from the names in the paper. When in doubt, please consult the source code documentation for the unit, synapse, and plant models.

In [ ]:
# Please set the Draculab source folder below
%cd /home/z/projects/draculab/

In [ ]:
from draculab import *
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle

In [ ]:
# Configure the simulation

np.random.seed(123456) # always the same random values
#np.random.seed() # arbitrary random seed

# The following configuration settings should be clear to readers of
# the paper's Supplementary information

use_syne = False # whether to use the version of the network with "synergies"
par_set = 1 # parameter set. Either 1 or 2.
rot_SPF = True # whether to use mixed SPF inputs in M
new_gain = 1. # gain multiplier during the center-out reaching phase
new_noise = 0. # noise during center-out reaching (cf. C_sigma below)

n_reaches_l = 16 # number of random targets presented for initial learning
t_pres_l = 40. # target presentation time during initial learning
r = .1 # distance from target to center during radial reaching
t_pres = 5. # target presentation time for center-out reaching
n_trgs = 8 # number of targets for center-out reaching
n_rounds = 6 # number of times the n_trgs targets will be presented

# It's probably best not to play with the settings below
par_heter = 0.01 # amplitude of distribution for heterogeneous parameters
set_C_delay = False # whether set C_cid using analytical approach
rand_targets = True # whether to train using a large number of random targets
C_noise = True # whether C units are noisy (use euler_maru integrator)
M__C_rand_w = True # whether to randomly intialize weights for the M__C connections
permute_targets = True # whether to permute target order during radial reaches
M_size = 12 # number of units in the M population
SPF_size = 12 # number of units in the SPF population

ad = {'new_noise': new_noise,
      'new_gain': new_gain }

In [ ]:
# The two parameter sets
ps1={'A__M_lrate': 20.0,
     'A__M_w_max_frac': 0.34,
     'A__M_w_sum': 1.0,
     'AL_thresh': 0.56,
     'b_e': 1.,
     'C__C_antag': 1.6,
     'C__C_p_antag': 0.15,
     'C__C_p_syne': 0.26,
     'C__C_syne': 1.1,
     'C_adapt_amp': 0.0,
     'C_cid': 0.17,
     'C_sigma': 0.5,
     'C_slope': 2.25,
     'C_tau': 0.24,
     'C_tau_slow': 2.0,
     'C_thresh': 1.14,
     'CE__CI_w': 0.39,
     'CI__CE_w': -1.8,
     'g_e_03': 20.,
     'CI_slope': 3.9,
     'CI_tau': 0.06,
     'CI_thresh': 1.37,
     'g_e_factor': 3.,
     'II_g_03': 2.73,
     'M__C_lrate': 500.0,
     'M__C_w_sum': 3.28,
     'M__M_w': 0.0,
     'M_cid': 1.,
     'M_des_out_w_abs_sum': 1.87,
     'M_tau': 0.012,
     'SF_thresh_03': 0.59,
     'SPF__SPF_w': -1.6,
     'fitness': None,
     'n_evals': 0,
     't_pres': t_pres_l,
     'par_heter': par_heter}

ps2={'A__M_lrate': 20.0,
     'A__M_w_max_frac': 0.33,
     'A__M_w_sum': 1.0,
     'AL_thresh': 0.55,
     'b_e': 1.1,
     'C__C_antag': 1.7,
     'C__C_p_antag': 0.27,
     'C__C_p_syne': 0.33,
     'C__C_syne': 1.13,
     'C_adapt_amp': 0.,
     'C_cid': 0.15,
     'C_sigma': 0.47,
     'C_slope': 2.26,
     'C_tau': 0.22,
     'C_tau_slow': 2.0,
     'C_thresh': 1.05,
     'CE__CI_w': 0.38,
     'CI__CE_w': -1.89,
     'g_e_03': 21.1,
     'CI_slope': 3.8,
     'CI_tau': 0.025,
     'CI_thresh': 1.34,
     'g_e_factor': 3.07,
     'II_g_03': 2.9,
     'M__C_lrate': 500.,
     'M__C_w_sum': 3.4,
     'M__M_w': 0.0,
     'M_cid': 1.1,
     'M_des_out_w_abs_sum': 2.,
     'M_tau': 0.033,
     'SF_thresh_03': 0.55,
     'SPF__SPF_w': -1.77,
     'fitness': None,
     'n_evals': 0,
     't_pres': t_pres_l,
     'par_heter': par_heter}

if par_set == 1:
    cfg = ps1
elif par_set == 2:
    cfg = ps2
else:
    raise ValueError('The value of par_set should be 1 or 2')

In [ ]:
# Create the network
%cd /home/z/projects/draculab/notebook/spinal/
from v3ft3p2ph2_net import *
%cd ../..

if not use_syne:
    net, pops_dict, hand_coords, m_idxs, pds = net_from_cfg(cfg,
                                                       t_pres = t_pres_l,
                                                       par_heter = par_heter,
                                                       set_C_delay = set_C_delay,
                                                       rand_targets = rand_targets,
                                                       track_weights = True,
                                                       track_ips = True,
                                                       C_noise = C_noise,
                                                       M__C_rand_w = M__C_rand_w,
                                                       rot_SPF = rot_SPF)
else:
    net, pops_dict, hand_coords, m_idxs, pds = syne_net(cfg,
                                                   t_pres = t_pres_l,
                                                   par_heter = par_heter,
                                                   set_C_delay = set_C_delay,
                                                   rand_targets = rand_targets,
                                                   track_weights = True,
                                                   track_ips = True,
                                                   C_noise = C_noise,
                                                   M__C_rand_w = M__C_rand_w,
                                                   rot_SPF = rot_SPF)

for name in pops_dict:
    exec("%s = %s"% (name, str(pops_dict[name])))

# parameter dictionaries used to change targets
P__A_syn = pds['P__A_syn']
P__A_ws = np.array(P__A_syn['init_w'][12:18])
A__SF_syn = pds['A__SF_syn']
SF_params = pds['SF_params']

In [ ]:
# Run simulation for initial learning
sim_time_l = t_pres_l * n_reaches_l
start_wctime = time.time()
times_l, data_l, plant_data_l  = net.flat_run(sim_time_l)

data_l = np.array(data_l)
print('Execution time is %s seconds' % (time.time() - start_wctime))

In [ ]:
# Produce figures

tic_s = 25 # tick label size
tit_s = 25 # title size
leg_s = 15 # legend text size
lab_s = 25 # axis label font size
arm_activs = plant_data_l[P]

u_to_disp = [0,1,2,6,8,9] # units to display
# SPF
redSPF = [SPF[u] for u in u_to_disp] # "reduced" SPF
fs = (25,6)
SPF_fig = plt.figure(figsize=fs)
SPF_ax = plt.gca()
SPF_ax.tick_params(axis='both', labelsize=tic_s)
SPF_data_l = np.array(data_l[redSPF])
plt.plot(times_l, SPF_data_l.transpose())
for rch in range(n_reaches_l): # plot a line at the onset of new targets
    plt.plot([t_pres_l*rch, t_pres_l*rch], [-0.01, 1.], 'y--', linewidth=1)
#SPF_legends = ['$S_{PA}$'+str(i) for i in range(len(redSPF))]
SPF_legends = ['$S_{PA}$'+str(i) for i in u_to_disp]
plt.legend(SPF_legends, fontsize=15)
plt.title('$S_{PA}$', fontsize=tit_s)

# M
redM = [M[u] for u in u_to_disp] # "reduced" M
M_fig = plt.figure(figsize=fs)
M_data_l = np.array(data_l[redM])
plt.plot(times_l, M_data_l.transpose())
M_legends = ['$M_{'+str(i)+'}$' for i in u_to_disp]
for rch in range(n_reaches_l): # plot a line at the onset of new targets
    plt.plot([t_pres_l*rch, t_pres_l*rch], [-0.01, 1.], 'y--', linewidth=1)
plt.legend(M_legends, fontsize=15)
plt.title('M', fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

if use_syne:
    # SYNE, SYNI
    n_syn_f = 4 # how many unit pairs to plot
    SYN_fig = plt.figure(figsize=fs)
    SYNE_data_l = np.array(data_l[SYNE[:n_syn_f]])
    SYNI_data_l = np.array(data_l[SYNI[:n_syn_f]])
    plt.plot(times_l, SYNE_data_l.transpose(), linewidth=2)
    plt.plot(times_l, SYNI_data_l.transpose(), '--')
    SYN_legends = ['SYNE'+str(i) for i in range(n_syn_f)]
    SYN_legends += ['SYNI'+str(i) for i in range(n_syn_f)]
    plt.legend(SYN_legends, fontsize=leg_s)
    plt.title('SYN', fontsize=tit_s)
    plt.xlabel('time [s]', fontsize=lab_s)
    plt.xticks(fontsize=tic_s)
    plt.yticks(fontsize=tic_s)
    for rch in range(n_reaches_l): # plot a line at the onset of new targets
        plt.plot([t_pres_l*rch, t_pres_l*rch], [-0.5, 1.5], 'y--', linewidth=1)
    
    # M--SYNE0 weights
    W_fig1 = plt.figure(figsize=fs)
    w_track_data_l = np.array(data_l[M_SYNE0_track])
    plt.plot(times_l, w_track_data_l.transpose())
    M_SYNE0_legends = ['M'+str(i)+'--SYNE0' for i in range(len(M_SYNE0_track))]
    plt.legend(M_SYNE0_legends)
    plt.title('M--SYNE0 weights', fontsize=tit_s)
    plt.xticks(fontsize=tic_s)
    plt.yticks(fontsize=tit_s)
    for rch in range(n_reaches_l): # plot a line at the onset of new targets
        plt.plot([t_pres_l*rch, t_pres_l*rch], [-0.01, 1.5], 'y--', linewidth=1)
else:
    # C
    cu_to_disp = [0] # units to display
    redCE = [CE[u] for u in cu_to_disp]
    redCI = [CI[u] for u in cu_to_disp] 
    C_fig = plt.figure(figsize=fs)
    CE_data_l = np.array(data_l[redCE])
    CI_data_l = np.array(data_l[redCI])
    plt.plot(times_l, CE_data_l.transpose(), linewidth=2)
    plt.plot(times_l, CI_data_l.transpose(), '--')
    C_legends = ['CE'+str(i) for i in cu_to_disp]
    C_legends += ['CI'+str(i) for i in cu_to_disp]
    plt.legend(C_legends, fontsize=leg_s)
    plt.title('C', fontsize=tit_s)
    plt.xlabel('time [s]', fontsize=lab_s)
    plt.xticks(fontsize=tic_s)
    plt.yticks(fontsize=tic_s)
    for rch in range(n_reaches_l): # plot a line at the onset of new targets
        plt.plot([t_pres_l*rch, t_pres_l*rch], [-0.5, 1.5], 'y--', linewidth=1)

    # M--CE0 weights
    W_fig1 = plt.figure(figsize=fs)
    w_track_data_l = np.array(data_l[M_CE0_track])
    plt.plot(times_l, w_track_data_l.transpose())
    M_CE0_legends = ['M'+str(i)+'--C0' for i in range(len(M_CE0_track))]
    plt.legend(M_CE0_legends, fontsize=15)
    plt.title('M--C0 weights', fontsize=tit_s)
    plt.xlabel('time [s]', fontsize=lab_s)
    plt.xticks(fontsize=tic_s)
    plt.yticks(fontsize=tic_s)
    for rch in range(n_reaches_l): # plot a line at the onset of new targets
        plt.plot([t_pres_l*rch, t_pres_l*rch], [-0.01, 1.5], 'y--', linewidth=1)

# AF--M0 weights
W_fig2 = plt.figure(figsize=fs)
w_track_data2_l = np.array(data_l[A_M0_track[0:18]])
plt.plot(times_l, w_track_data2_l.transpose())
A_M0_legends = ['A'+str(i)+'--M0' for i in range(len(A_M0_track[:12]))]
plt.legend(A_M0_legends, fontsize=15)
plt.title('A--M0 weights exc', fontsize=tit_s)
plt.xlabel('time [s]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
for rch in range(n_reaches_l): # plot a line at the onset of new targets
    plt.plot([t_pres_l*rch, t_pres_l*rch], [-0.01, .4], 'y--', linewidth=1)

plt.show()

In [ ]:
# P
P_fig = plt.figure(figsize=fs)
P_state = plant_data_l[P][:,0:4:2]
plt.plot(times_l, P_state)
#plt.legend(['sh ang', 'sh ang vel', 'elb ang', 'elb ang vel'])
plt.legend(['sh ang', 'elb ang',], fontsize=leg_s)
plt.title('double pendulum state variables', fontsize=tit_s)
#plt.xlabel('time [s]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
for rch in range(n_reaches_l): # plot a line at the onset of new targets
    plt.plot([t_pres_l*rch, t_pres_l*rch], [-0.5, 2.5], 'y--', linewidth=1)

# A
A_fig = plt.figure(figsize=fs)
A_data_l = np.array(data_l[A[0:18]])
plt.plot(times_l, A_data_l.transpose())
A_legends = ['Ib' + str(i) for i in range(6)] + \
            ['Ia' + str(i) for i in range(6)] + \
            ['II' + str(i) for i in range(6)]
plt.legend(A_legends, fontsize=12)
plt.title('A', fontsize=tit_s)
#plt.xlabel('time [s]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

#fs = (30,6)
# SF, SP
SF_fig, axs = plt.subplots(1, 3, figsize=(fs[0], fs[1]))
for ax in axs.flatten(): 
    ax.tick_params(axis='both', labelsize=tic_s)
SF_data_l = np.array(data_l[SF])
SP_data_l = np.array(data_l[SP])
if 'lowpass_SP' in locals() and lowpass_SP is True:
    LPF_SP_data_l = np.array(data_l[LPF_SP])
for col in range(3):
    ax = axs[col]
    base = 2*col
    ax.plot(times_l, SF_data_l[base:base+2, :].transpose(), linewidth=2)
    ax.plot(times_l, SP_data_l[base:base+2, :].transpose(), '--', linewidth=1)
    if 'lowpass_SP' in locals() and lowpass_SP is True:
        ax.plot(times_l, LPF_SP_data_l[base:base+2, :].transpose(), linewidth=2)
    ax.set_title('SF, SP, units %d to %d' % (base, base+1), fontsize=tit_s)
    SF_legends = ['SF '+ str(base+i) for i in range(2)]
    SP_legends = ['SP '+ str(base+i) for i in range(2)]
    ax.legend(SF_legends + SP_legends, fontsize=leg_s)

plt.show()

In [ ]:
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")
    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")
    if window_len<3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y


plant = net.plants[P]
# modified copy-paste of plt.upd_ip_impl
q1 = arm_activs[:,0]
q2 = arm_activs[:,2]
q12 = q1+q2
c_elbow = np.array((plant.l_arm*np.cos(q1), plant.l_arm*np.sin(q1)))
c_hand = np.array((c_elbow[0] + plant.l_farm*np.cos(q12),
                   c_elbow[1] + plant.l_farm*np.sin(q12))).transpose()
if 'start_t' in locals():
    coord_idxs = np.floor((times_l-start_t)/t_pres_l).astype(int) # after resetting the targets
    des_coords = np.array([hand_coords[idx] for idx in [m_idxs[cid] for cid in coord_idxs]])
else:
    coord_idxs = np.floor(times_l/t_pres_l).astype(int)  # before resetting the targets
    des_coords = np.array(hand_coords)[m_idxs[coord_idxs],:] # desired coordinates at each moment in time
coords_fig = plt.figure(figsize=fs)
plt.plot(times_l, c_hand)
plt.plot(times_l, des_coords)
plt.title('desired vs. actual hand coordinates', fontsize=tit_s)
plt.legend(['X', 'Y', 'des_X', 'des_Y'], fontsize=leg_s)
#plt.xlabel('time [s]', fontsize=lab_s)
plt.ylabel('coordinate [m]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

act_fig = plt.figure(figsize=fs)
act_data_l = np.array(data_l[ACT])[0]
plt.plot(times_l, act_data_l)
plt.plot(times_l, 0.8*np.ones_like(times_l), 'k--')
plt.title('ACT', fontsize=tit_s)
plt.xlabel('time [s]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

err_fig = plt.figure(figsize=fs)
w_len = 1001
hand_error = np.linalg.norm(c_hand-des_coords, axis=1)
smooth_hand_error = smooth(hand_error, window_len=w_len)[int(np.floor(w_len/2)):-int(np.floor(w_len/2))]
#plt.plot(times_l, smooth_hand_error)
plt.plot(times_l, hand_error, 'r')
plt.plot(times_l, 0.1+np.zeros(smooth_hand_error.size), 'k--')
plt.title('distance from target', fontsize=tit_s)
for rch in range(n_reaches_l): # plot a line at the onset of new targets
    plt.plot([t_pres_l*rch, t_pres_l*rch], [-0.01, 0.6], 'y--', linewidth=1)
#plt.xlabel('time [s]', fontsize=lab_s)
plt.ylabel('error [m]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

avg_error = hand_error.sum()/hand_error.size
print("average error: %f" % (avg_error))


plt.show()

In [ ]:
# Alpha units
fs2 =(25,5)
AL_fig = plt.figure(figsize=fs2)
AL_data = np.array(data_l[AL])
plt.plot(times_l, AL_data.transpose())
AL_legends = ['AL'+str(i) for i in range(len(AL))]
plt.legend(AL_legends)
plt.title('AL')
print('AL_data:')
print(AL_data[:,-1])

# plotting muscle outputs
fs2 =(25,5)
legs = ['Ib', 'Ia', 'II']

for i in range(6):
    next_fig = plt.figure(figsize=fs2)
    Ib = arm_activs[:,22+i]
    Ia = arm_activs[:,28+i]
    II = arm_activs[:,34+i]
    plt.plot(times_l, Ib, times_l, Ia, times_l, II)
    plt.legend(legs)
    plt.title('m' + str(i))
    print('Ib avg for muscle '+ str(i) + '= ' + str(np.mean(Ib)))
    print('Ia avg for muscle '+ str(i) + '= ' + str(np.mean(Ia)))
    print('II avg for muscle '+ str(i) + '= ' + str(np.mean(II)))
plt.show()

In [ ]:
# power spectrum
pop_id = M[0]
init_idx = 20000
from spectrum import *
fig1 = plt.figure()
spec_data = data_l[pop_id][init_idx:init_idx+8192]
spec_data = spec_data - np.mean(spec_data)
p = Periodogram(spec_data, sampling=(1./net.min_delay))
p.plot(marker='o')
plt.xlim([0,10])

fig2 = plt.figure()
res = pmtm(spec_data, NW=2.5, k=4, show=True)
plt.xlim([0,800])

In [ ]:
def set_new_targets(net,
                    pops_dict,
                    r,
                    t_pres,
                    n_trgs,
                    n_rounds,
                    pds,
                    permute_targets=True):
    """ Set new targets for reaching.
        
        Args:
            net: the draculab network
            pops_dict: dictionary with unit IDs for each population
            r: distance from center to targets
            t_pres: new presentation time
            n_trgs: number of targets
            n_rounds: number of times the n_trgs targets will be presented
            pds : parameter dictionaries used to calculate SP values.
            permute_targets: (boolean) are targets presented permuted?

            
        Returns:
            hand_coords : list with target coordinates (each a numpy 2-array).
            targets : list, only with targets
            center : only the center coordinates
            trg_ids: target used for each presentation
    """
    
    start_t = net.sim_time # starting time for new simulation
    # 8 radial targets in sequence, from 0 to 315 degrees
    r = 0.1 # distance from center to targets
    center = np.array([0.3, 0.3]) # initial hand location
    angs = np.linspace(0., 2.*np.pi, n_trgs+1)[:-1]
    circle = np.array([np.array([np.cos(ang),np.sin(ang)]) for ang in angs])
    targets = center + r*circle # coordinates of the targets

    if permute_targets:
        # version with permuted targets, all are seen every 8 presentations
        trg_ids = np.random.permutation(n_trgs*n_rounds)%n_trgs # target for each presentation
        hand_coords = []
        for idx in trg_ids:
            hand_coords += [center, targets[idx]]
        hand_coords += [center] # to avoid indexes out of bounds
    else:    
        # version with sequential targets
        hand_coords = [center, targets[0],
                       center, targets[1],
                       center, targets[2],
                       center, targets[3],
                       center, targets[4],
                       center, targets[5],
                       center, targets[6],
                       center, targets[7]]
        hand_coords = n_rounds * hand_coords # many repetitions of the same sequence
        hand_coords += [center] # to avoid indexes out of bounds
        trg_ids = np.arange(len(hand_coords))%n_trgs # target for each presentation

    SP = pops_dict['SP']
    A = pops_dict['A']
    P = pops_dict['P']
    #### next is a copy-pasta of the code to set the SP values
    # list with muscle lengths corresponding to the hand coordinates
    m_lengths = []
    for coord in hand_coords:
        m_lengths.append(net.plants[P].coords_to_lengths(coord))
    m_lengths = np.array(m_lengths)
    #(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)
    # We need to translate these lengths to corresponding SF activity levels.
    # For that it is necessary to recreate all their transformations
    # The first transformation is from length to II afferent activity.
    ### OUT OF THE 36 AFFERENT SIGNALS, WE TAKE II ###
    par = net.plants[P].m_params
    # steady state tensions in the static and dynamic bag fibers (no gamma inputs)
    Ts_ss = (par['k_se_s']/(par['k_se_s']+par['k_pe_s'])) * (
             par['k_pe_s']*(m_lengths - par['l0_s']))
    Td_ss = (par['k_se_d']/(par['k_se_d']+par['k_pe_d'])) * (
             par['k_pe_d']*(m_lengths - par['l0_d']))
    # steady state afferent outputs (no gamma inputs)
    #Ia_ss = par['fs']*(Ts_ss/par['k_se_s']) + (1.-par['fs'])*(Td_ss/par['k_se_d'])
    II_ss = par['se_II']*(Ts_ss/par['k_se_s']) + ((1.-par['se_II'])/par['k_pe_s'])*Ts_ss
    #Ia_ss *= par['Ia_gain']
    II_ss *= par['II_gain']
    #Ia_II_ss = np.concatenate((Ia_ss, II_ss), axis=1)
    # Next transformation is through the afferent units
    P__A_ws = np.array(pds['P__A_syn']['init_w'][12:18])
    #Ia_II_avgs = np.mean(Ia_II_ss, axis=0)  # when using hundreds of random targets
    # target averages
    A_thr = np.array([net.units[u].thresh for u in A[12:18]])
    A_II = np.log(1. + np.maximum((II_ss)*P__A_ws - A_thr, 0.))
    #(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)(.)
    # Next is from A to SF
    SF_arg = pds['A__SF_syn']['init_w']*A_II
    SF_params = pds['SF_params']
    SF_out = 1./ (1. + np.exp(-SF_params['slope']*(SF_arg - SF_params['thresh'])))
    SF_params['init_val'] = SF_out # this might cause a smooth start
    # now we set the values in SP
    #m_idxs = np.random.randint(len(hand_coords), size=1000) # index of all targets
    m_idxs = list(range(len(hand_coords)+1)) # reach list targets sequentially
        #m_idxs[0] = 0 # for testing
    A_us = [net.units[u] for u in A]

    def SF_sigmo(idx, arg):
        """ The sigmoidal function for SF unit with index SF[idx]. """
        return 1./ (1. + np.exp(-SF_params['slope'][idx]*(arg - SF_params['thresh'][idx])))

    def cur_target(t):
        """ Returns the index of the target at time t. """
        return m_idxs[int(np.floor((t-start_t)/t_pres))]

    def make_fun(idx):
        """ create a function for the SP unit with index 'idx'. """
        return lambda t: SF_sigmo(idx, 
                            pds['A__SF_syn']['init_w'][idx] * (
                            np.log(1. + max(II_ss[cur_target(t)][idx] * P__A_ws[idx] - 
                            net.units[A[12+idx]].thresh, 0.))))

    for idx, u in enumerate(SP):
        net.units[u].set_function(make_fun(idx))
        
    return hand_coords, targets, center, trg_ids, m_idxs

In [ ]:
# set new targets with the function

start_t = net.sim_time # starting time for new simulation
hand_coords, targets, center, trg_ids, m_idxs = set_new_targets(net, 
                                                          pops_dict,
                                                                  r, 
                                                             t_pres,
                                                             n_trgs, 
                                                            n_rounds,
                                                            pds)

In [ ]:
# remove noise from C units
if use_syne:
    for u in [net.units[c] for c in SYNE+SYNI]:
        u.sigma = ad['new_noise']    
else:
    for u in [net.units[c] for c in CE+CI]:
        u.sigma = ad['new_noise']
    
# increase the muscle gains
for idx in range(6):
    net.plants[0].inp_syns[idx][0].w = ad['new_gain']

In [ ]:
# Run simulation for radial reaches
sim_time2 = 2 * t_pres * n_trgs * n_rounds
start_wctime = time.time()
times, data, plant_data  = net.flat_run(sim_time2)
data = np.array(data)
print('Execution time is %s seconds' % (time.time() - start_wctime))

In [ ]:
# Produce figures

tic_s = 25 # tick label size
tit_s = 25 # title size
leg_s = 15 # legend text size
lab_s = 25 # axis label font size
arm_activs = plant_data[P]

u_to_disp = [0,1,2,6,8,9] # units to display
# SPF
redSPF = [SPF[u] for u in u_to_disp] # "reduced" SPF
fs = (25,6)
SPF_fig = plt.figure(figsize=fs)
SPF_ax = plt.gca()
SPF_ax.tick_params(axis='both', labelsize=tic_s)
SPF_data = np.array(data[redSPF])
plt.plot(times, SPF_data.transpose())
# for rch in range(n_reaches): # plot a line at the onset of new targets
#     plt.plot([t_pres*rch, t_pres*rch], [-0.01, 1.], 'y--', linewidth=1)
#SPF_legends = ['$S_{PA}$'+str(i) for i in range(len(redSPF))]
SPF_legends = ['$S_{PA}$'+str(i) for i in u_to_disp]
plt.legend(SPF_legends, fontsize=15)
plt.title('$S_{PA}$', fontsize=tit_s)

# M
redM = [M[u] for u in u_to_disp] # "reduced" M
M_fig = plt.figure(figsize=fs)
M_data = np.array(data[redM])
plt.plot(times, M_data.transpose())
M_legends = ['$M_{'+str(i)+'}$' for i in u_to_disp]
# for rch in range(n_reaches): # plot a line at the onset of new targets
#     plt.plot([t_pres*rch, t_pres*rch], [-0.01, 1.], 'y--', linewidth=1)
plt.legend(M_legends, fontsize=15)
plt.title('M', fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

if use_syne:
    # SYNE, SYNI
    n_syn_f = 4 # how many unit pairs to plot
    SYN_fig = plt.figure(figsize=fs)
    SYNE_data = np.array(data[SYNE[:n_syn_f]])
    SYNI_data = np.array(data[SYNI[:n_syn_f]])
    plt.plot(times, SYNE_data.transpose(), linewidth=2)
    plt.plot(times, SYNI_data.transpose(), '--')
    SYN_legends = ['SYNE'+str(i) for i in range(n_syn_f)]
    SYN_legends += ['SYNI'+str(i) for i in range(n_syn_f)]
    plt.legend(SYN_legends, fontsize=leg_s)
    plt.title('SYN', fontsize=tit_s)
    plt.xlabel('time [s]', fontsize=lab_s)
    plt.xticks(fontsize=tic_s)
    plt.yticks(fontsize=tic_s)
#     for rch in range(n_reaches): # plot a line at the onset of new targets
#         plt.plot([t_pres*rch, t_pres*rch], [-0.5, 1.5], 'y--', linewidth=1)
    
    # M--SYNE0 weights
    W_fig1 = plt.figure(figsize=fs)
    w_track_data = np.array(data[M_SYNE0_track])
    plt.plot(times, w_track_data.transpose())
    M_SYNE0_legends = ['M'+str(i)+'--SYNE0' for i in range(len(M_SYNE0_track))]
    plt.legend(M_SYNE0_legends)
    plt.title('M--SYNE0 weights', fontsize=tit_s)
    plt.xticks(fontsize=tic_s)
    plt.yticks(fontsize=tit_s)
#     for rch in range(n_reaches): # plot a line at the onset of new targets
#         plt.plot([t_pres*rch, t_pres*rch], [-0.01, 1.5], 'y--', linewidth=1)
else:
    # C
    cu_to_disp = [0] # units to display
    redCE = [CE[u] for u in cu_to_disp]
    redCI = [CI[u] for u in cu_to_disp] 
    C_fig = plt.figure(figsize=fs)
    CE_data = np.array(data[redCE])
    CI_data = np.array(data[redCI])
    plt.plot(times, CE_data.transpose(), linewidth=2)
    plt.plot(times, CI_data.transpose(), '--')
    C_legends = ['CE'+str(i) for i in cu_to_disp]
    C_legends += ['CI'+str(i) for i in cu_to_disp]
    plt.legend(C_legends, fontsize=leg_s)
    plt.title('C', fontsize=tit_s)
    plt.xlabel('time [s]', fontsize=lab_s)
    plt.xticks(fontsize=tic_s)
    plt.yticks(fontsize=tic_s)
#     for rch in range(n_reaches): # plot a line at the onset of new targets
#         plt.plot([t_pres*rch, t_pres*rch], [-0.5, 1.5], 'y--', linewidth=1)

    # M--CE0 weights
    W_fig1 = plt.figure(figsize=fs)
    w_track_data = np.array(data[M_CE0_track])
    plt.plot(times, w_track_data.transpose())
    M_CE0_legends = ['M'+str(i)+'--C0' for i in range(len(M_CE0_track))]
    plt.legend(M_CE0_legends, fontsize=15)
    plt.title('M--C0 weights', fontsize=tit_s)
    plt.xlabel('time [s]', fontsize=lab_s)
    plt.xticks(fontsize=tic_s)
    plt.yticks(fontsize=tic_s)
#     for rch in range(n_reaches): # plot a line at the onset of new targets
#         plt.plot([t_pres*rch, t_pres*rch], [-0.01, 1.5], 'y--', linewidth=1)

# AF--M0 weights
W_fig2 = plt.figure(figsize=fs)
w_track_data2 = np.array(data[A_M0_track[0:18]])
plt.plot(times, w_track_data2.transpose())
A_M0_legends = ['A'+str(i)+'--M0' for i in range(len(A_M0_track[:12]))]
plt.legend(A_M0_legends, fontsize=15)
plt.title('A--M0 weights exc', fontsize=tit_s)
plt.xlabel('time [s]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
# for rch in range(n_reaches): # plot a line at the onset of new targets
#     plt.plot([t_pres*rch, t_pres*rch], [-0.01, .4], 'y--', linewidth=1)

plt.show()

In [ ]:
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")
    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")
    if window_len<3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y


plant = net.plants[P]
# modified copy-paste of plt.upd_ip_impl
q1 = arm_activs[:,0]
q2 = arm_activs[:,2]
q12 = q1+q2
c_elbow = np.array((plant.l_arm*np.cos(q1), plant.l_arm*np.sin(q1)))
c_hand = np.array((c_elbow[0] + plant.l_farm*np.cos(q12),
                   c_elbow[1] + plant.l_farm*np.sin(q12))).transpose()
if 'start_t' in locals():
    coord_idxs = np.floor((times-start_t+1e-8)/t_pres).astype(int) # after resetting the targets
    des_coords = np.array([hand_coords[idx] for idx in [m_idxs[cid] for cid in coord_idxs]])
else:
    coord_idxs = np.floor(times/t_pres).astype(int)  # before resetting the targets
    des_coords = np.array(hand_coords)[m_idxs[coord_idxs],:] # desired coordinates at each moment in time
coords_fig = plt.figure(figsize=fs)
plt.plot(times, c_hand)
plt.plot(times, des_coords)
plt.title('desired vs. actual hand coordinates', fontsize=tit_s)
plt.legend(['X', 'Y', 'des_X', 'des_Y'], fontsize=leg_s)
plt.xlabel('time [s]', fontsize=lab_s)
plt.ylabel('coordinate [m]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

act_fig = plt.figure(figsize=fs)
act_data = np.array(data[ACT])[0]
plt.plot(times, act_data)
plt.plot(times, 0.8*np.ones_like(times), 'k--')
plt.title('ACT', fontsize=tit_s)
plt.xlabel('time [s]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

err_fig = plt.figure(figsize=fs)
w_len = 1001
hand_error = np.linalg.norm(c_hand-des_coords, axis=1)
smooth_hand_error = smooth(hand_error, window_len=w_len)[int(np.floor(w_len/2)):-int(np.floor(w_len/2))]
plt.plot(times, smooth_hand_error)
plt.plot(times, hand_error, 'r--')
plt.plot(times, 0.1+np.zeros(smooth_hand_error.size), 'k--')
plt.title('distance from target', fontsize=tit_s)
avg_error = hand_error.sum()/hand_error.size
print("average error: %f" % (avg_error))
plt.xlabel('time [s]', fontsize=lab_s)
plt.ylabel('error [m]', fontsize=lab_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

plt.show()

In [ ]:
# Functions to arrange the multiunit data from the various reaches
def get_tensor(series, J=n_trgs, K=n_rounds, T=int(round(t_pres / net.min_delay))):
    """ Given a simulation time series, arrange it by reach and repetition.
    
        Args:
            series : a time series, spanning simtime2 seconds, in min_delay steps.
            J : number of targets in the simulation.
            K : number of repetitions of each target.
            T : number of time steps that each target is presented.
            
        Returns:
            tensor : a 3-dimensional array. tensor(j,k,t) is the value of the series
                     for target j, on the k-th repetition, at time step t. Time steps
                     begin anew for each reach.
    """
    tensor = np.zeros((J, K, T))
    print(tensor.shape)
    trg_reps = np.zeros(J, dtype=int) # how many repetitions of each target we have filled

    for reach, trg in enumerate(trg_ids):
        init_tid = (1 + 2 * reach) * T
        final_tid = init_tid + T # t_pres seconds later
        tensor[trg, trg_reps[trg],:] = series[init_tid:final_tid]
        trg_reps[trg] += 1
        
    return tensor

def get_pop_tensor(pop, J=n_trgs, K=n_rounds, t_smp =.5, t_strt=0., trg_ids=trg_ids):
    """ Returns a tensor with the activities in a population arranged by target and repetition.
    
        Args:
            pop : a list with the IDs of the populations units in the 'data' matrix.
            J : number of targets in the simulation.
            K : number of repetitions of each target.
            t_smp : time to sample per reach (in seconds)
            t_strt : time after target onset when sample begins.
            trg_ids : trg_ids[i] is the target at the i-th reach
        Returns:
            tensor : a 4-dimensional array. tensor(i,j,k,t) is the activity of the i-th
                     unit, when reaching target j, on the k-th repetition, at time step t.
    """
    idx_strt = int(t_strt / net.min_delay)
    I = len(pop)
    T=int(t_smp / net.min_delay) # number of time steps that each target is presented.
    tensor = np.zeros((I,J,K,T), dtype=np.float_)

    trg_reps = np.zeros(J, dtype=int) # how many repetitions of each target we have filled
    pt_per_pres = int(round(t_pres / net.min_delay)) # t_pres should be an argument!
    for reach, trg in enumerate(trg_ids):
        init_tid = (1 + 2 * reach) * pt_per_pres + idx_strt
        final_tid = init_tid + int(t_smp/net.min_delay) # t_smp seconds later
        tensor[:, trg, trg_reps[trg],:] = data[pop, init_tid:final_tid]
        trg_reps[trg] += 1
        
    return tensor

In [ ]:
# Arrange the error data into a tensor
# he_tensor[j,k,t] is the hand tensor for target j,
# o the k-th presentation, at time t
J = n_trgs # number of targets
K = n_rounds # number of repetitions per target
T = int(round(t_pres / net.min_delay)) # number of time points

he_tensor = get_tensor(hand_error, J=J, K=K, T=T)
    
# we average across repetitions of the same target
ahe_trg = he_tensor.sum(axis=1) / K

# we average across all reaches
ahe = ahe_trg.sum(axis=0) / J

In [ ]:
# Print distance to target for each target
ahe_fig = plt.figure(figsize=fs)
reach_ts = np.linspace(0., t_pres, T)
plt.plot(reach_ts, ahe, linewidth=4)
plt.title("Distance to target (average across all targets)", fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

n_rows = 2
n_cols = 4
ahe_trg_fig, ahe_trg_axs = plt.subplots(n_rows, n_cols, figsize=(fs[0], 2*fs[1]), sharey=True)
for row in range(n_rows):
    for col in range(n_cols):
        ax = ahe_trg_axs[row,col]
        trg = row*n_cols + col
        ax.plot(reach_ts, ahe_trg[trg, :], linewidth=4)
        ax.tick_params(axis='both', labelsize=tic_s)
        ax.set_title("target " + str(trg), fontsize=tit_s)
        for rep in range(K):
            ax.plot(reach_ts, he_tensor[trg,rep,:], '--', linewidth=1, color='b')

plt.show()

In [ ]:
# Obtain hand coordinates through time
from numpy import cos, sin
if not 'targets' in locals():
    targets = np.array([[0.3, 0.3]])

# angles of shoulder and elbow
theta_s = arm_activs[:,0]
theta_e = arm_activs[:,2]
phi = theta_s + theta_e # elbow angle wrt x axis
# data from tracking units
ipx = data[ipx_track,:]
ipy = data[ipy_track,:]
ten = arm_activs[:, np.array(range(4,10))].transpose()
# coordinates of hand and elbow
l1 = net.plants[P].l_arm
l2 = net.plants[P].l_farm
xe = cos(theta_s)*l1
ye = sin(theta_s)*l1
xh = xe + cos(phi)*l2
yh = ye + sin(phi)*l2

# placing hand coordinates in tensors (arrange by target, repetition)
xh_tensor = get_tensor(xh, J=J, K=K, T=T)
yh_tensor = get_tensor(yh, J=J, K=K, T=T)
    
# average across reptitions of the same target
axh_trg = xh_tensor.sum(axis=1) / K
ayh_trg = yh_tensor.sum(axis=1) / K

In [ ]:
# Obtain velocity profile for the reaches
xph = np.concatenate(([0], (xh[1:] - xh[:-1]) / net.min_delay))
yph = np.concatenate(([0], (yh[1:] - yh[:-1]) / net.min_delay))
hv = np.sqrt(xph*xph + yph*yph)

hv_tensor = get_tensor(hv)

# we average across repetitions of the same target
ahv_trg = hv_tensor.sum(axis=1) / K

# we average across all reaches
ahv = ahv_trg.sum(axis=0) / J

In [ ]:
# Print velocity for each target
ahv_fig = plt.figure(figsize=fs)
reach_ts = np.linspace(0., t_pres, T)
plt.plot(reach_ts, ahe, linewidth=4)
plt.plot(reach_ts, ahv,  'r', linewidth=4)
#plt.title("hand velocity (average across all targets)", fontsize=tit_s)
plt.title("Average across all targets", fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
#plt.ylabel('velocity [m/s]', fontsize=lab_s)
plt.xlabel('time [s]', fontsize=lab_s)
plt.legend(['error [m]', 'speed [m/s]'], fontsize=leg_s)

n_rows = 2
n_cols = 4
ahv_trg_fig, ahv_trg_axs = plt.subplots(n_rows, n_cols, figsize=(fs[0], 2*fs[1]), sharey=True)
for row in range(n_rows):
    for col in range(n_cols):
        ax = ahv_trg_axs[row,col]
        trg = row*n_cols + col
        ax.plot(reach_ts, ahv_trg[trg, :], 'r', linewidth=4)
        ax.tick_params(axis='both', labelsize=tic_s)
        ax.set_title("target " + str(trg), fontsize=tit_s)
        for rep in range(K):
            ax.plot(reach_ts, hv_tensor[trg,rep,:], '--', linewidth=1, color='red')
        if col==0:
            ax.set_ylabel('velocity [m/s]', fontsize=lab_s)

plt.show()

In [ ]:
# plot average reach trajectories and some individual reaches
n_ind_reaches = 6 # number of example reaches to plot
reach_fig = plt.figure(figsize=(10,10))
reach_ax = plt.gca()
trial = 0
reach_ax.scatter(targets[:,0], targets[:,1], s=1800, c='cyan')
# reach_ax.scatter(center[0]+.8*(targets[:,0]-center[0]), 
#                 center[1]+.8*(targets[:,1]-center[1]), s=1800, c='k', alpha=0.3)
#reach_ax.scatter(.6*(targets[:,0]+center[0]), .6*(targets[:,1]+center[1]), s=1000, c='cyan', alpha=0.4)
reach_ax.scatter(center[0], center[1], s=2000, c='blue')
reach_ax.tick_params(axis='both', labelsize=tic_s)
reach_ax.set_xlabel('X [m]', fontsize=lab_s)
reach_ax.set_ylabel('Y [m]', fontsize=lab_s)
reach_ax.set_xticks([0.2, 0.25, 0.3, 0.35, 0.4])
reach_ax.set_xticks([0.2, 0.25, 0.3, 0.35, 0.4])
plt.axis('equal')
reach_ax.grid()
cmap = [[0.1, 0.4, 0.8, 0.5],
        [0.9, 0.7, 0.2, 0.8],
        [0.1, 0.7, 0.1, 0.6],
        [0.9, 0.1, 0.1, 0.5],
        [0.6, 0.2, 0.6, 0.5],
        [0.8, 0.4, 0.1, 0.5],
        [0.7, 0.1, 0.7, 0.5],
        [0.3, 0.3, 0.3, 0.5]]
for trg in range(J):
    reach_ax.plot(axh_trg[trg,:], ayh_trg[trg,:], linewidth=3)
    for rep in range(min(n_ind_reaches, K)):
        reach_ax.plot(xh_tensor[trg, rep, :], yh_tensor[trg, rep, :], '--', linewidth=1, color=cmap[trg])
#from IPython.display import Image
#Image(filename='temp_fig')
#plt.savefig('temp_fig2')
reach_ax.set_xlim([0.16, .44])
plt.show()

In [ ]:
# power spectrum
pop_id = M[0]
from spectrum import *
fig1 = plt.figure()
spec_data = data[pop_id][0:8192]
spec_data = spec_data - np.mean(spec_data)
p = Periodogram(spec_data, sampling=(1./net.min_delay))
p.plot(marker='o')
plt.xlim([0,10])

fig2 = plt.figure()
res = pmtm(spec_data, NW=2.5, k=4, show=True)
plt.xlim([0,800])

---
### Direction tuning analysis
---

In [ ]:
# Obtaining the prefered directions for the units in M
# Find the mean rate for each target
t_smp = 4. # time to sample M (in seconds)
I = 12 # number of units to consider in M
m_tensor = get_pop_tensor(M[:I], t_smp=t_smp)
# m_tensor[i,j,k,t] is the activity of the i-th M 
# unit for target j, on the k-th repetition, at time point t.

# Now we average across repetitions for the same target
m_tensor_trg_avg = m_tensor.sum(axis=2) / K
# m_tensor_trg_avg[i,j,t]: average activity of i-th M unit when
# reaching for target j, at time point t

m_avg_rates = np.zeros((I,J)) # average rate at each unit for each target
for trg in range(J):
    m_avg_rates[:, trg] = np.mean(m_tensor_trg_avg[:, trg, :], axis=1)

m_means = np.mean(m_avg_rates, axis=1) # mean rate for each unit
    
# Obtaining preferred directions by fitting the rates using a plane, and using
# the angle of the projection of this plane's normal vector onto the XY plane
# The coefficients of the normal vector can be calculated using the least-squares
# method, which leads to a 3x3 linear system that is readily reduced to a 2x2 system.
trgs = targets - center # targets centered at the origin
xs = trgs[:,0]
ys = trgs[:,1]
cxs = xs - np.mean(xs) # "centered" x coordinates
cys = ys - np.mean(ys) #  "centered" Y coordinates

X11 = (xs * cxs).sum()
X12 = (xs * cys).sum()
X21 = (ys * cxs).sum()
X22 = (ys * cys).sum()

Amat = np.array([[X11, X12], [X21, X22]])
detA = np.linalg.det(Amat) # determinant of Amat
if detA == 0.:
    raise ValueError('Indeterminate system found!')
invA = np.linalg.inv(Amat)

prf_angs = np.zeros(len(M)) # preferred angles, in radians
normal_vecs = [] # list with the vectors normal to the plane fitting the rates

print("Coeffs. of determination for PD vectors.")
for uid in range(I):
    r1 = (xs * (m_avg_rates[uid, :] - m_means[uid])).sum()
    r2 = (ys * (m_avg_rates[uid, :] - m_means[uid])).sum()
    n = np.matmul(invA, np.array([r1,r2]))
    # boils down to
    #n = [a,b] where b = r2/X22, a = r1/X11
    #normal_vecs.append(n / np.linalg.norm(n)) # appending normalized vector
    normal_vecs.append(n)
    prf_angs[uid] = np.arctan2(n[1], n[0]) # preferred angle
    #print("n1=%f, n0=%f, prf_ang=%f"%(n[1],n[0]))

    c = np.mean(m_avg_rates[uid,:] - n[0]*xs - n[1]*ys)
    
    # obtaining residuals, coefficient of determination, R^2
    residuals = m_avg_rates[uid,:] - n[0]*xs - n[1]*ys - c
    devs = m_avg_rates[uid,:] - m_means[uid]
    SSr = (residuals * residuals).sum()
    SSt = (devs * devs).sum()
    R = 1. - (SSr/SSt)
    print(R)

normal_vecs = np.array(normal_vecs)

# Get the lengths of the preferred direction vectors
pd_norms = np.linalg.norm(normal_vecs, axis=1)

In [ ]:
# Obtaining predicted PDs

def ang_diff(a1, a2):
    """ Difference between two angles in the range [-pi, pi]. """
    big_a = max(a1,a2)
    lil_a = min(a1,a2)
    if lil_a < 0 and big_a > 0:
        diff = min(np.pi-big_a - lil_a, big_a-lil_a)
    else:
        diff = big_a - lil_a
    return diff

# len_diffs[i,j] is the required length of contraction for muscle i, target j, divided by the length in the rest position
# This is obtained in 'source_code_2'
len_diffs = np.array( [[0.        , 0.        , 0.00512814, 0.02957183, 0.03340439, 0.01043197, 0.        , 0.        ],
                       [0.02868621, 0.13750184, 0.16861731, 0.12856412, 0.02810964, 0.        , 0.        , 0.        ],
                       [0.        , 0.        , 0.        , 0.        , 0.        , 0.10604558, 0.16349133, 0.10088526],
                       [0.0308563 , 0.01576455, 0.        , 0.        , 0.        , 0.        , 0.01655006, 0.03257554],
                       [0.        , 0.        , 0.        , 0.        , 0.09134384, 0.13319637, 0.09134384, 0.        ],
                       [0.11777032, 0.16048272, 0.11777032, 0.0165468 , 0.        , 0.        , 0.        , 0.0165468 ]])

ldiff_vecs = np.zeros((6,2))
ldiff_angs = np.zeros(6)
angs = np.linspace(0., 2.*np.pi, J+1)[:-1]
for idx, lds in enumerate(len_diffs):
    ldiff_vecs[idx,0] = (lds*np.cos(angs)).sum()
    ldiff_vecs[idx,1] = (lds*np.sin(angs)).sum()
    ldiff_angs[idx] = np.arctan2(ldiff_vecs[idx,1], ldiff_vecs[idx,0])

# obtain the connection matrix
# SPF__M_ws[i,j]: weight from j-th SPF unit to i-th M unit
SPF__M_ws = np.zeros((len(M), len(SPF)))
if rot_SPF:
    for m_idx, m in enumerate(M):
        w_list = [syn.w for syn in net.syns[m] if syn.preID in SPF]
        SPF__M_ws[m_idx, :] = w_list
else:
    SPF__M_ws = np.eye(12)
    
# Obtain predicted PD vectors
pred_pds = np.zeros((12,2)) # predicted PD vectors for the 12 M units
for m_idx, m_ws in enumerate(SPF__M_ws):
    for idx, vec in enumerate(ldiff_vecs):
        pred_pds[m_idx,:] += (m_ws[idx]-m_ws[idx+6]) * vec
        
# angle and norm of the PD vectors
pred_angs = np.zeros(12)
pred_norms = np.zeros(12)
for idx, pd in enumerate(pred_pds):
    pred_angs[idx] = np.arctan2(pd[1], pd[0])
    pred_norms[idx] = np.linalg.norm(pd)
    
# Coefficients of determination for the predicted angles ("unweighted" version)
pred_residuals1 = np.array([ang_diff(a1,a2) for a1,a2 in zip(pred_angs, prf_angs)])
ang_devs1 = np.array([ang_diff(a1, np.mean(prf_angs)) for a1 in prf_angs])
SSr1 = (pred_residuals1 * pred_residuals1).sum()
SSt1 = (ang_devs1 * ang_devs1).sum()
R1 = 1. - (SSr1/SSt1)

# Coefficients of determination for the predicted angles ("weighted" version)
pred_residuals2 = np.array([ang_diff(a1,a2) for a1,a2 in zip(pred_angs, prf_angs)])
pred_residuals2 = pred_residuals2 * pd_norms
ang_devs2 = np.array([ang_diff(a1, np.mean(prf_angs)) for a1 in prf_angs])
ang_devs2 = ang_devs2 * pd_norms
SSr2 = (pred_residuals2 * pred_residuals2).sum()
SSt2 = (ang_devs2 * ang_devs2).sum()
R2 = 1. - (SSr2/SSt2)

print("unweighted R: %f, weighted R: %f" % (R1, R2))

# Plot direction tuning for all M units
n_rows = 2
n_cols = 6
dir_fig, dir_axs = plt.subplots(n_rows, n_cols, figsize=(1.7*fs[0], 2.4*fs[1]), subplot_kw=dict(projection='polar'))

# m_avg_rates[i,j]: average rate of i-th M unit 
# when reaching for target j
span = max(m_avg_rates.flatten()) * max(trgs.flatten())
theta = np.linspace(0.0, 2 * np.pi, J, endpoint=False)
width = np.pi / J
for row in range(n_rows):
    for col in range(n_cols):
        ax = dir_axs[row][col]
        m_id = n_cols*row + col
        ax.set_title("M = %d" % (m_id), fontsize=20) 
        ax.bar(theta, m_avg_rates[m_id, :], width=width, bottom=0.0, color='b', alpha=0.3)
        #ax.bar(prf_angs[m_id], .5*pd_norms[m_id], width=np.pi/32, bottom=0.0, color='r')
        ax.arrow(prf_angs[m_id],.01, 0., .35*pd_norms[m_id], width=0.07,
             head_width=0.15, head_length=0.1, length_includes_head=True, color='r')
        ax.arrow(pred_angs[m_id],.01, 0., .35*pred_norms[m_id], width=0.03,
                 head_width=0.1, head_length=0.07, length_includes_head=True, color='k')
        ax.tick_params(axis='x', labelsize=22)
        ax.tick_params(axis='y', labelsize=18)
        if use_syne:
            ax.set_yticks([.1, .3, .5])
        else:
            ax.set_yticks([.1, .25, .5])
                    
plt.show()

In [ ]:
# Plot per-target mean activity for first A units

at_smp = 2. # time to sample M (in seconds)
at_strt =1.
aI = 6 # number of units to consider in A
a_tensor = get_pop_tensor(A[:aI], J=J, K=K, t_smp=at_smp, t_strt=at_strt)
a_tensor_trg_avg = a_tensor.sum(axis=2) / K
angs = np.linspace(0., 2.*np.pi, J+1)[:-1]

a_avg_rates = np.zeros((aI,J)) # average rate at each unit for each target
for trg in range(J):
    a_avg_rates[:, trg] = np.mean(a_tensor_trg_avg[:, trg, :], axis=1)

# obtain average vector
a_vecs = np.zeros((6,2))
a_angs = np.zeros(6)
for idx, arat in enumerate(a_avg_rates):
    a_vecs[idx,0] = (arat*np.cos(angs)).sum()
    a_vecs[idx,1] = (arat*np.sin(angs)).sum()
    a_angs[idx] = np.arctan2(a_vecs[idx,1], a_vecs[idx,0])
a_norms = np.linalg.norm(a_vecs, axis=1)

dir_fig, dir_axs = plt.subplots(1, 6, figsize=(1.7*fs[0], fs[1]), subplot_kw=dict(projection='polar'), sharey=True)

# a_avg_rates[i,j]: average rate of i-th M unit 
# when reaching for target j
span = max(a_avg_rates.flatten()) * max(trgs.flatten())
theta = np.linspace(0.0, 2 * np.pi, J, endpoint=False)
width = np.pi / J
for col in range(aI):
    ax = dir_axs[col]
    a_id = col
    ax.set_title("A = %d" % (a_id), fontsize=20) 
    ax.bar(theta, a_avg_rates[a_id, :], width=width, bottom=0.0, color='r', alpha=0.3)
    ax.arrow(a_angs[a_id],.01, 0., 0.5*a_norms[a_id], width=0.07,
         head_width=0.15, head_length=0.1, length_includes_head=True, color='k')
    ax.tick_params(axis='x', labelsize=22)
    ax.tick_params(axis='y', labelsize=18)
    ax.set_yticks([.2, .4, .6])
                     
plt.show()

To assess how many units are "significantly tuned" to reach direction we can use a bootstrap method. Basically, you shuffle the identities of the targets used (the `trg_ids` array), and calculate the length of the preferred direction (PD) vector (`normal_vecs`) for each cell. After producing $N$ different permutations of the targets, you have $N$ PD vectors for each unit. A cell is significantly tuned if the length of its PD vector is larger than $.999N$ of the PD vectors.

This is based on [Scott and Kalaska 1997](https://journals.physiology.org/doi/full/10.1152/jn.1997.77.2.826).

In [ ]:
# Assess if significantly tuned to direction

N_shuffles = 10000 # number of target permutations to produce
sh_pd_norms = np.zeros((I, N_shuffles)) # magnitude of PD vector for each cell, all shuffles
#m_rates_tensor = np.mean(m_tensor, axis=3)
# m_rates_tensor[i,j,k]: mean firing rate for cell i on k-th presentation of target j
for n_idx in range(N_shuffles):
    sh_trg_ids = np.random.permutation(trg_ids)
    sh_m_tensor = get_pop_tensor(M[:I], J=J, K=K, t_smp=t_smp, trg_ids=sh_trg_ids)
    sh_m_tensor_trg_avg = sh_m_tensor.sum(axis=2) / K
    sh_m_avg_rates = np.zeros((I,J)) # average rate at each unit for each target
    for trg in range(J):
        sh_m_avg_rates[:, trg] = np.mean(sh_m_tensor_trg_avg[:, trg, :], axis=1)
    sh_m_means = np.mean(sh_m_avg_rates, axis=1) # mean rate for each unit
    sh_normal_vecs = np.zeros((I,2))
    for uid in range(I):
        r1 = (xs * (sh_m_avg_rates[uid, :] - sh_m_means[uid])).sum()
        r2 = (ys * (sh_m_avg_rates[uid, :] - sh_m_means[uid])).sum()
        n = np.matmul(invA, np.array([r1,r2]))
        sh_normal_vecs[uid,:] = n
    sh_pd_norms[:, n_idx] = np.linalg.norm(sh_normal_vecs, axis=1)
    
# Now, for each unit, find vlue of sh_pd_norms that is larger than 99%
sig_threshs = np.percentile(sh_pd_norms, 99.9, axis=1)

# For each unit, check if it is significantly tuned
tuned = pd_norms >= sig_threshs
print(tuned)
n_tuned = tuned.sum()
print(n_tuned)

In [ ]:
# plot of preferred directions for the 12 cells

pd_scaling = True # whether to scale angle contributions by length of PD vector
# Find the angle of the dominant axis of the skew (bimodal version)
mod_prf_angs = [ang if ang > 0. else ang+np.pi for ang in prf_angs]
dom_ang = np.arctan2(np.sin(mod_prf_angs).sum(), np.cos(mod_prf_angs).sum())
# if the contribution of each angle is scaled by the norm of its PD vector we get sc_dom_ang
sc_dom_ang = np.arctan2((np.sin(mod_prf_angs)*pd_norms).sum(), (np.cos(mod_prf_angs)*pd_norms).sum())
dom_ang_deg = dom_ang * 180./np.pi # angle of dominant axis in degrees
sc_dom_ang_deg = sc_dom_ang * 180. / np.pi

if pd_scaling:
    d_a = sc_dom_ang
    d_a_deg = sc_dom_ang_deg
else:
    d_a = dom_ang
    d_a_deg = dom_ang_deg
    
print("Angle of dominant axis: %f" %(d_a_deg))

#sig_prf_angs = prf_angs
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=fs)
#ax.plot(prf_angs, np.ones(len(prf_angs)), '*')
#ax.plot(prf_angs, pd_norms, '^')
for uid in range(len(prf_angs)):
    ax.arrow(prf_angs[uid],.001, 0., pd_norms[uid], width=0.01,
             head_width=0.1, head_length=0.2, length_includes_head=True)
#ax.bar(prf_angs, np.ones(len(prf_angs)), width=np.pi/40, color='b', alpha=0.3)
#ax.bar(prf_angs, pd_norms, width=np.pi/40, alpha=0.3)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=15)
ax.set_yticks([.5, 1., 1.5])
ax.set_rmax(1.1*max(pd_norms))
ax.plot([d_a, d_a], [0., 1.05*max(pd_norms)], 'k--', alpha=0.5)
ax.plot([d_a+np.pi, d_a+np.pi], [0., 1.05*max(pd_norms)], 'k--', alpha=0.5)
ax.plot([d_a+np.pi/4., d_a+np.pi/4.], [0., 1.05*max(pd_norms)], 'r--')
ax.plot([d_a+np.pi+np.pi/4., d_a+np.pi+np.pi/4.], [0., 1.05*max(pd_norms)], 'r--')
plt.show()


In [ ]:
# plot of preferred directions for the 12 cells, only ones with large norm
big_norms_idx = np.arange(len(pd_norms))[pd_norms > np.mean(pd_norms)]
sig_prf_angs = prf_angs[big_norms_idx]
#sig_prf_angs = prf_angs[tuned]
mod_sig_prf_angs = [ang if ang > 0. else ang+np.pi for ang in sig_prf_angs]
# Find the angle of the dominant axis of the skew (bimodal version)
sig_dom_ang = np.arctan2(np.sin(mod_sig_prf_angs).sum(), np.cos(mod_sig_prf_angs).sum())
sig_dom_ang_deg = sig_dom_ang * 180./np.pi # angle of dominant axis in degrees
print("Angle of dominant axis: %f" %(sig_dom_ang_deg))
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(sig_prf_angs, np.ones(len(sig_prf_angs)), '*')
ax.bar(sig_prf_angs, np.ones(len(sig_prf_angs)), width=np.pi/40, color='b', alpha=0.3)
ax.set_rmax(1.1*max(pd_norms))
ax.plot([sig_dom_ang, sig_dom_ang], [0., 1.1], 'k--', alpha=0.5)
ax.plot([sig_dom_ang+np.pi, sig_dom_ang+np.pi], [0., 1.1], 'k--', alpha=0.5)
plt.show()

In [ ]:
# Obtaining unimodal and bimodal Rayleigh r statistics
# These are twice the value of those in Lillicrap and Scott 2013 for the unimodal case
r_unimod = 2. * ((np.sin(prf_angs).sum())**2 + (np.cos(prf_angs).sum())**2) / I
r_bimod =  ((np.sin(2.*prf_angs).sum())**2 + (np.cos(2.*prf_angs).sum())**2) / I

# Significance threshold for unimodal r
a = .99
x_c = np.sqrt(-2.*np.log(1 - a))

# Significance threshold for bimodal r
n_samples = 100000 # how many times the r statistic is randomly produced
rnd_angles = 2.*np.pi*np.random.random((n_samples, I)) # 10000 random angles sets in (0,2pi)
rnd_r_bim = (np.sin(2.*rnd_angles).sum(axis=1)**2 + 
             np.cos(2.*rnd_angles).sum(axis=1)**2) / I

sig_r_thresh = np.percentile(rnd_r_bim, 99.)

print("r_unimod=%f, r_bimod=%f, x_c=%f, x_c_bim=%f" % (r_unimod, r_bimod, x_c, sig_r_thresh))

In [ ]:
# Obtaining unimodal and bimodal Rayleigh r statistics, big norms
# These are twice the value of those in Lillicrap and Scott 2013
r_unimod = 2. * ((np.sin(sig_prf_angs).sum())**2 + (np.cos(sig_prf_angs).sum())**2) / sig_prf_angs.size
r_bimod = ((np.sin(2.*sig_prf_angs).sum())**2 + (np.cos(2.*sig_prf_angs).sum())**2) / sig_prf_angs.size

# Significance threshold
a = .99
x_c = np.sqrt(-2.*np.log(1 - a))

# Significance threshold for bimodal r
n_samples = 100000 # how many times the r statistic is randomly produced
rnd_angles = 2.*np.pi*np.random.random((n_samples, sig_prf_angs.size))
rnd_r_bim = (np.sin(2.*rnd_angles).sum(axis=1)**2 + 
             np.cos(2.*rnd_angles).sum(axis=1)**2) / sig_prf_angs.size

sig_r_thresh = np.percentile(rnd_r_bim, 99.)

print("r_unimod=%f, r_bimod=%f, x_c=%f, x_c_bim=%f" % (r_unimod, r_bimod, x_c, sig_r_thresh))
bimodal = r_bimod > x_c
print(bimodal)

---
### Circular dynamics analysis
---

In [ ]:
# jPCA analysis part 1

# the first step is to obtain m_tensor
t_smp = 1. # time to sample M (in seconds) t_smp=.6, t_strt=1.
t_strt = 1.1 # time after target onset when sample begins
normalize=False
idx_strt = int(t_strt / net.min_delay)

I = 12 #12 # number of dimensions to consider in M
# J = n_trgs # number of targets
# K = n_rounds # number of repetitions per target
T = int(t_smp / net.min_delay) # number of time points

m_tensor = get_pop_tensor(M[:I], J=J, K=K, t_smp=t_smp, t_strt=t_strt)

# Normalize responses in m_tensor
if normalize:
    for i in range(I):
        for j in range(J):
            for k in range(K):
                norm = np.linalg.norm(m_tensor[i,j,k,:])
                m_tensor[i,j,k,:] = m_tensor[i,j,k,:] / norm
    
# Now we average across repetitions for the same target
p_tensor = m_tensor.sum(axis=2) / K

# We obtain the across-condition average
a_tensor = p_tensor.sum(axis=1) / J

# Obtained a normalized average trace per condition
c_tensor = np.zeros_like(p_tensor)
for trg in range(J):
    c_tensor[:, trg, :] = p_tensor[:, trg, :] - a_tensor

In [ ]:
munit = 0
# trg_fig = plt.figure(figsize=fs)
# plt.plot(times[:T], m_tensor[munit,5,0,:])

M_fig = plt.figure(figsize=fs)
# M_data = np.array(data[M])
initl = int(4.*6./net.min_delay)
plt_strt = initl + int(t_strt / net.min_delay)
plt_stop = initl + int(t_strt / net.min_delay) + T
plt.plot(times[initl:plt_stop+50], M_data[:, initl:plt_stop+50].transpose(), linewidth=2)
plt.plot([times[plt_strt], times[plt_strt]], [0.1, .7], 'k--')
plt.plot([times[plt_stop], times[plt_stop]], [0.1, .7], 'k--')
plt.title("M firing rates", fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
plt.xlabel('Simulation time [s]', fontsize=lab_s)
#plt.plot(times[0:T], data[M[0], 0:T])

c_fig = plt.figure(figsize=fs)
plt.plot(np.linspace(0., t_smp, T), c_tensor[:,6,:].transpose())
#plt.plot([times[plt_strt], times[plt_strt]], [-0.015, .02], 'k--')
#plt.plot([times[plt_stop], times[plt_stop]], [-0.015, .02], 'k--')
plt.title("Normalized average M traces", fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
plt.xlabel('Sample time [s]', fontsize=lab_s)

p_fig = plt.figure(figsize=fs)
plt.plot(times[:T], p_tensor[:,6,:].transpose())
plt.title("p")

a_fig = plt.figure(figsize=fs)
plt.plot(times[:T], a_tensor[:,:].transpose())
plt.title("a")

plt.show()

In [ ]:
# jPCA analysis part 2

import scipy as sp
# Obtain the plane of strongest rotation

X = np.zeros((J*T,I))
for j in range(J):
    X[j*T:(j+1)*T, :] = c_tensor[:,j,:].transpose()

# create the block-matrix version of X
Xtilde = sp.linalg.block_diag(*([X]*I))

# Create the H matrix
n = X.shape[1]
ct = X.shape[0]
L = np.zeros((n,n), dtype=int)
c = 0
for row in range(n):
    for col in range(row+1, n):
        L[row, col] = c
        L[col, row] = c
        c += 1
        
H = np.zeros((n*n, int(0.5*n*(n-1))))
for col in range(n):
    for row in range(n):
        if col > row:
            H[n*col+row, L[col,row]] = 1.
        elif row > col:
            H[n*col+row, L[col,row]] = -1.
            
# Approximate the derivatives of X
Xp = np.zeros_like(X)
t_bit = times[1] - times[0]
Xp[1:,:] = (X[1:,:] - X[:-1,:]) / t_bit

xp = Xp.flatten('F')

kstar = np.matmul(np.linalg.pinv(np.matmul(Xtilde, H)), xp)

# reconstruct the matrix that generated the data
Mstar = np.matmul(H, kstar).reshape(n,n)

# Next,extract the eigenvalues of M.
eig_vals, eig_vecs = np.linalg.eig(Mstar)
# print(Mstar)
# print("Eigenvalues: ")
# print(eig_vals)
# print("Eigenvectors: ")
# print(eig_vecs)

eig_vals_norms = np.sqrt((eig_vals * eig_vals.conj()).real)
ev_sum = eig_vals_norms.sum()/2.
var_percentages = eig_vals_norms[np.array([0,2,4,6,8,10])] / ev_sum
print("Variance percentages by each conjugate eigenvalue pair")
print(var_percentages)

# We obtain the plane of strongest rotations
srt_idxs = np.argsort(-eig_vals_norms) # indexes for sorting in decreasing order
srt_eig_vecs = eig_vecs[:, srt_idxs]
u1 = (srt_eig_vecs[:,0] + srt_eig_vecs[:,1]).real
u2 = (1j * (srt_eig_vecs[:,0] - srt_eig_vecs[:,1])).real
u1 = u1 / np.linalg.norm(u1)
u2 = u2 / np.linalg.norm(u2)


In [ ]:
# jPCA analysis part 3

#n_pts = int(t_smp / net.min_delay)
n_pts = min(100, T-1) # number of points in the trace to plot
# project activities into the plane of strongest rotations
jPCA1_traces = np.zeros((J, T), dtype=np.float_)
jPCA2_traces = np.zeros((J, T), dtype=np.float_)
u1 = u1.reshape(1,I)
u2 = u2.reshape(1,I)

for target in range(J):
    jPCA1_traces[target, :] = np.matmul(u1, c_tensor[:,target,:])
    jPCA2_traces[target, :] = np.matmul(u2, c_tensor[:,target,:])

    
# RGB colormap. First entry is greenest, last is reddest
cmap = [[0., .9, 0.],
        [0., .7, 0.],
        [0., .5, 0.],
        [0., .2, 0.],
        [.4, 0., 0.],
        [.5, 0., 0.],
        [.7, 0., 0.],
        [.9, 0., 0.]]
t0_jPC1s = [jPCA1_traces[target,0] for target in range(J)]
jPC1_sortr = list(np.argsort(t0_jPC1s))
#t0_jPC1s.sort() # sorting according to initial jPC1 component
jpca_fig =  plt.figure(figsize=(12,12))
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
y_span = [min(jPCA2_traces.flatten()), max(jPCA2_traces.flatten())]
x_span = [min(jPCA1_traces.flatten()), max(jPCA1_traces.flatten())]
plt.plot([0.,0.], y_span, 'k', linewidth=1)
plt.plot(x_span, [0., 0.], 'k', linewidth=1)
plt.xlabel('$jPCA_1$', fontsize=lab_s)
plt.ylabel('$jPCA_2$', fontsize=lab_s)
for target in range(J):
    c = cmap[jPC1_sortr.index(target)]
    plt.plot(jPCA1_traces[target,:n_pts], jPCA2_traces[target,:n_pts], color=c, linewidth=4)
    plt.scatter(jPCA1_traces[target,0], jPCA2_traces[target,0], s=200, color=c)
    dx = jPCA1_traces[target,n_pts] - jPCA1_traces[target,n_pts-5]
    dy = jPCA2_traces[target,n_pts] - jPCA2_traces[target,n_pts-5]
    plt.arrow(jPCA1_traces[target,n_pts-5],jPCA2_traces[target,n_pts-5],
         dx, dy, color=c, width=0.001, head_width=0.005)

plt.show()

In [ ]:
# jPCA part 4
# Other measures of rotation

# 1) average angle between X and Xp
X_norms = np.linalg.norm(X, axis=1)
Xp_norms = np.linalg.norm(Xp, axis=1)
X_cos = (X * Xp).sum(axis=1) / (X_norms * Xp_norms + 1e-10)
off = 0
#X_cos = (np.roll(X,off,axis=0)[off:] * Xp[:-off]).sum(axis=1) / (np.roll(X_norms,off,axis=0)[off:] * Xp_norms[:-off] + 1e-10)
#X_cos = (X[off:] * Xp[:-off]).sum(axis=1) / (X_norms[off:] * Xp_norms[:-off] + 1e-10)
X_angs = np.arccos(X_cos)

wid = 0.4 # widgth of window around pi/2 where X,Xp are "orthogonal"
orth_fr = ((X_angs < np.pi/2+wid) & (X_angs > np.pi/2-wid)).sum() / len(X_angs)
print("orthogonal fraction:%f" % (orth_fr))
n_bins = 30
hist_fig = plt.figure(figsize=fs)
plt.hist(X_angs, bins=n_bins)
plt.xticks(ticks=[0., np.pi/4, np.pi/2, 3.*np.pi/4., np.pi],
          labels=['0', '$\pi/4$', '$\pi/2$', '$3 \pi/4$', '$\pi$'], fontsize=tic_s)
plt.yticks(fontsize=tic_s)
plt.xlabel('angle [rads]', fontsize=lab_s)
plt.ylabel('count', fontsize=lab_s)
plt.show()

# 2) Coefficients of determination
# 2.1) Obtain unconstrained M matrix
M_uncons = np.matmul(np.linalg.pinv(X), Xp)
# 2.2) Reconstruct Xp with M_uncons
Xp_uncons = np.matmul(X, M_uncons)
# 2.3) Reconstruct Xp with Mstar
Xp_skew = np.matmul(X, Mstar)
# 2.4) Calculate residual sums of squares
SSres_uncons = ((Xp - Xp_uncons) * (Xp - Xp_uncons)).sum()
SSres_skew = ((Xp - Xp_skew) * (Xp - Xp_skew)).sum()
# 2.5) Calculate the total sum of squares
SStot = ((Xp-Xp.mean())*(Xp-Xp.mean())).sum()
# 2.6) Calculate coefficients of determination
R2_uncons = 1. - (SSres_uncons / SStot)
R2_skew = 1. - (SSres_skew / SStot)

print("R2 unconstrained: %f" % (R2_uncons))
print("R2 skew symmetric: %f" % (R2_skew))

---
## Second direction tuning analysis (PD drift)
---

**Warning**: depending on your computer, this simulation may take several hours.

In [ ]:
# set new targets with the function

n_rounds = 40
n_trgs = 10 # more targets!

start_t = net.sim_time # starting time for new simulation
hand_coords, targets, center, trg_ids, m_idxs = set_new_targets(net, 
                                                          pops_dict,
                                                                  r, 
                                                             t_pres,
                                                             n_trgs, 
                                                           n_rounds,
                                                                pds)

In [ ]:
# Run simulation for radial reaches
sim_time2 = 2 * t_pres * n_trgs * n_rounds
start_wctime = time.time()
times, data, plant_data  = net.flat_run(sim_time2)

data = np.array(data)
print('Execution time is %s seconds' % (time.time() - start_wctime))

In [ ]:
# Obtain hand error
arm_activs = plant_data[P]
q1 = arm_activs[:,0]
q2 = arm_activs[:,2]
q12 = q1+q2
c_elbow = np.array((plant.l_arm*np.cos(q1), plant.l_arm*np.sin(q1)))
c_hand = np.array((c_elbow[0] + plant.l_farm*np.cos(q12),
                   c_elbow[1] + plant.l_farm*np.sin(q12))).transpose()
if 'start_t' in locals():
    coord_idxs = np.floor((times-start_t+1e-8)/t_pres).astype(int) # after resetting the targets
    des_coords = np.array([hand_coords[idx] for idx in [m_idxs[cid] for cid in coord_idxs]])
else:
    coord_idxs = np.floor(times/t_pres).astype(int)  # before resetting the targets
    des_coords = np.array(hand_coords)[m_idxs[coord_idxs],:] # desired coordinates at each moment in time
    
hand_error = np.linalg.norm(c_hand-des_coords, axis=1)

In [ ]:
# Arrange the error data into a tensor
# he_tensor[j,k,t] is the hand tensor for target j,
# o the k-th presentation, at time t
J = n_trgs # number of targets
K = n_rounds # number of repetitions per target
T = int(round(t_pres / net.min_delay)) # number of time points

he_tensor = get_tensor(hand_error, J=J, K=K, T=T)
# he_tensor = np.zeros((J, K, T))
# trg_reps = np.zeros(J, dtype=int) # how many repetitions of each target we have filled
# #pt_per_pres = int(round(t_pres / net.min_delay))
# for reach, trg in enumerate(trg_ids):
#     init_tid = (1 + 2 * reach) * T
#     final_tid = init_tid + T # t_pres seconds later
#     he_tensor[trg, trg_reps[trg],:] = hand_error[init_tid:final_tid]
#     trg_reps[trg] += 1
    
# we average across repetitions of the same target
ahe_trg = he_tensor.sum(axis=1) / K

# we average across all reaches
ahe = ahe_trg.sum(axis=0) / J

In [ ]:
# Print distance to target for each target
ahe_fig = plt.figure(figsize=fs)
reach_ts = np.linspace(0., t_pres, T)
plt.plot(reach_ts, ahe, linewidth=4)
plt.title("Distance to target (average across all targets)", fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)

n_rows = 2
n_cols = 4
ahe_trg_fig, ahe_trg_axs = plt.subplots(n_rows, n_cols, figsize=(fs[0], 2*fs[1]), sharey=True)
for row in range(n_rows):
    for col in range(n_cols):
        ax = ahe_trg_axs[row,col]
        trg = row*n_cols + col
        ax.plot(reach_ts, ahe_trg[trg, :], linewidth=4)
        ax.tick_params(axis='both', labelsize=tic_s)
        ax.set_title("target " + str(trg), fontsize=tit_s)
        for rep in range(K):
            ax.plot(reach_ts, he_tensor[trg,rep,:], '--', linewidth=1, color='b')

plt.show()

In [ ]:
# Obtain hand coordinates through time
from numpy import cos, sin
if not 'targets' in locals():
    targets = np.array([[0.3, 0.3]])

# angles of shoulder and elbow
theta_s = arm_activs[:,0]
theta_e = arm_activs[:,2]
phi = theta_s + theta_e # elbow angle wrt x axis
# data from tracking units
ipx = data[ipx_track,:]
ipy = data[ipy_track,:]
ten = arm_activs[:, np.array(range(4,10))].transpose()
# coordinates of hand and elbow
l1 = net.plants[P].l_arm
l2 = net.plants[P].l_farm
xe = cos(theta_s)*l1
ye = sin(theta_s)*l1
xh = xe + cos(phi)*l2
yh = ye + sin(phi)*l2

# placing hand coordinates in tensors (arrange by target, repetition)
xh_tensor = get_tensor(xh, J=J, K=K, T=T)
yh_tensor = get_tensor(yh, J=J, K=K, T=T)

# xh_tensor = np.zeros((J, K, T))
# yh_tensor = np.zeros((J, K, T))
# trg_reps = np.zeros(J, dtype=int) # how many repetitions of each target we have filled
# for reach, trg in enumerate(trg_ids):
#     init_tid = (1 + 2 * reach) * T
#     final_tid = init_tid + T
#     xh_tensor[trg, trg_reps[trg], :] = xh[init_tid:final_tid]
#     yh_tensor[trg, trg_reps[trg], :] = yh[init_tid:final_tid]
#     trg_reps[trg] += 1
    
# average across reptitions of the same target
axh_trg = xh_tensor.sum(axis=1) / K
ayh_trg = yh_tensor.sum(axis=1) / K

In [ ]:
# Obtain velocity profile for the reaches
xph = np.concatenate(([0], (xh[1:] - xh[:-1]) / net.min_delay))
yph = np.concatenate(([0], (yh[1:] - yh[:-1]) / net.min_delay))
hv = np.sqrt(xph*xph + yph*yph)

hv_tensor = get_tensor(hv, J=J, K=K, T=T)

# we average across repetitions of the same target
ahv_trg = hv_tensor.sum(axis=1) / K

# we average across all reaches
ahv = ahv_trg.sum(axis=0) / J

In [ ]:
# Print velocity for each target
ahv_fig = plt.figure(figsize=fs)
reach_ts = np.linspace(0., t_pres, T)
plt.plot(reach_ts, ahv, linewidth=4)
plt.title("hand velocity (average across all targets)", fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
plt.ylabel('velocity [m/s]', fontsize=lab_s)
plt.xlabel('time [s]', fontsize=lab_s)


n_rows = 2
n_cols = 4
ahv_trg_fig, ahv_trg_axs = plt.subplots(n_rows, n_cols, figsize=(fs[0], 2*fs[1]), sharey=True)
for row in range(n_rows):
    for col in range(n_cols):
        ax = ahv_trg_axs[row,col]
        trg = row*n_cols + col
        ax.plot(reach_ts, ahv_trg[trg, :], linewidth=4)
        ax.tick_params(axis='both', labelsize=tic_s)
        ax.set_title("target " + str(trg), fontsize=tit_s)
        for rep in range(K):
            ax.plot(reach_ts, hv_tensor[trg,rep,:], '--', linewidth=1, color='b')
        if col==0:
            ax.set_ylabel('velocity [m/s]', fontsize=lab_s)

plt.show()

In [ ]:
# plot average reach trajectories and some individual reaches
n_ind_reaches = 4 # number of example reaches to plot
reach_fig = plt.figure(figsize=(10,10))
reach_ax = plt.gca()
trial = 0
#reach_ax.scatter(targets[:,0], targets[:,1], s=1000, c='cyan')
reach_ax.scatter(center[0]+.8*(targets[:,0]-center[0]), 
                 center[1]+.8*(targets[:,1]-center[1]), s=1000, c='k', alpha=0.3)
#reach_ax.scatter(.6*(targets[:,0]+center[0]), .6*(targets[:,1]+center[1]), s=1000, c='cyan', alpha=0.4)
reach_ax.scatter(center[0], center[1], s=2000, c='blue')
plt.axis('equal')
reach_ax.grid()
cmap = [[0.1, 0.4, 0.8, 0.5],
        [0.2, 0.2, 0.2, 0.8],
        [0.9, 0.7, 0.2, 0.8],
        [0.1, 0.7, 0.1, 0.6],
        [0.9, 0.1, 0.1, 0.5],
        [0.6, 0.2, 0.6, 0.5],
        [0.8, 0.4, 0.1, 0.5],
        [0.7, 0.1, 0.7, 0.5],
        [0.5, 0.5, 0.2, 0.8],
        [0.3, 0.3, 0.3, 0.5]]
for trg in range(J):
    reach_ax.plot(axh_trg[trg,:], ayh_trg[trg,:], linewidth=3, color=cmap[trg])
    for rep in range(min(n_ind_reaches, K)):
        reach_ax.plot(xh_tensor[trg, rep, :], yh_tensor[trg, rep, :], '--', linewidth=1, color=cmap[trg])
#from IPython.display import Image
#Image(filename='temp_fig')
#plt.savefig('temp_fig2')
plt.show()

In [ ]:
# Obtaining the prefered directions for the units in M
# Find the mean rate for each target
t_smp = 1. # time to sample M (in seconds)
I = 12 # number of units to consider in M
m_tensor = get_pop_tensor(M[:I], J=J, K=K, t_smp=t_smp, trg_ids=trg_ids)
# m_tensor[i,j,k,t] is the activity of the i-th M 
# unit for target j, on the k-th repetition, at time point t.

# Now we average across repetitions for the same target
m_tensor_trg_avg = m_tensor.sum(axis=2) / K
# m_tensor_trg_avg[i,j,t]: average activity of i-th M unit when
# reaching for target j, at time point t

m_avg_rates = np.zeros((I,J)) # average rate at each unit for each target
for trg in range(J):
    m_avg_rates[:, trg] = np.mean(m_tensor_trg_avg[:, trg, :], axis=1)

m_means = np.mean(m_avg_rates, axis=1) # mean rate for each unit
    
# Obtaining preferred directions by fitting the rates using a plane, and using
# the angle of the projection of this plane's normal vector onto the XY plane
# The coefficients of the normal vector can be calculated using the least-squares
# method, which leads to a 3x3 linear system that is readily reduced to a 2x2 system.
trgs = targets - center # targets centered at the origin
xs = trgs[:,0]
ys = trgs[:,1]
cxs = xs - np.mean(xs) # "centered" x coordinates
cys = ys - np.mean(ys) #  "centered" Y coordinates

X11 = (xs * cxs).sum()
X12 = (xs * cys).sum()
X21 = (ys * cxs).sum()
X22 = (ys * cys).sum()

Amat = np.array([[X11, X12], [X21, X22]])
detA = np.linalg.det(Amat) # determinant of Amat
if detA == 0.:
    raise ValueError('Indeterminate system found!')
invA = np.linalg.inv(Amat)

prf_angsB = np.zeros(len(M)) # preferred angles, in radians
normal_vecsB = [] # list with the vectors normal to the plane fitting the rates

for uid in range(I):
    r1 = (xs * (m_avg_rates[uid, :] - m_means[uid])).sum()
    r2 = (ys * (m_avg_rates[uid, :] - m_means[uid])).sum()
    n = np.matmul(invA, np.array([r1,r2]))
    # boils down to
    #n = [a,b] where b = r2/X22, a = r1/X11
    #normal_vecsB.append(n / np.linalg.norm(n)) # appending normalized vector
    normal_vecsB.append(n)
    prf_angsB[uid] = np.arctan2(n[1], n[0]) # preferred angle
    #print("n1=%f, n0=%f, prf_ang=%f"%(n[1],n[0]))

    c = np.mean(m_avg_rates[uid,:] - n[0]*xs - n[1]*ys)
    
    # obtaining residuals, coefficient of determination, R^2
    residuals = m_avg_rates[uid,:] - n[0]*xs - n[1]*ys - c
    devs = m_avg_rates[uid,:] - m_means[uid]
    SSr = (residuals * residuals).sum()
    SSt = (devs * devs).sum()
    R = 1. - (SSr/SSt)
    print(R)

normal_vecsB = np.array(normal_vecsB)

# Get the lengths of the preferred direction vectors
pd_normsB = np.linalg.norm(normal_vecsB, axis=1)

In [ ]:
# Plot direction tuning for all M units

dir_fig, dir_axs = plt.subplots(3, 4, figsize=(fs[0], 3.3*fs[1]))


for row in range(3):
    for col in range(4):
        ax = dir_axs[row][col]
        m_id = 4*row + col
        span = max(np.linalg.norm(normal_vecs[m_id]), np.linalg.norm(normal_vecsB[m_id]))
        ax.set_xlim([-1.1*span, 1.1*span])
        ax.set_ylim([-1.1*span, 1.1*span])
        ax.grid()
        ax.set_title("M = %d" % (m_id), fontsize=tit_s)
        ax.tick_params(axis='both', labelsize=tic_s)
        mod_trgs = m_avg_rates[m_id,:].reshape(n_trgs,1) * trgs 
        #for trg in range(J):
            #ax.plot([trgs[trg][0]], [trgs[trg][1]], '*')
            #ax.plot([0., mod_trgs[trg, 0]], [0., mod_trgs[trg, 1]], linewidth=5, color='b')
        #nrms = np.linalg.norm(mod_trgs, axis=1)
#         ax.plot([0., normal_vecs[m_id, 0]], [0., normal_vecs[m_id, 1]], linewidth=3, color='r')
#         ax.plot([0., normal_vecsB[m_id, 0]], [0., normal_vecsB[m_id, 1]], linewidth=3, color='b')
        nrm = np.linalg.norm(normal_vecs[m_id,:])
        ax.arrow(0., 0., normal_vecs[m_id, 0], normal_vecs[m_id, 1],
                 width=.03*nrm, color='r', length_includes_head=True)
        ax.arrow(0., 0., normal_vecsB[m_id, 0], normal_vecsB[m_id, 1],
                 width=.03*nrm, color='b', length_includes_head=True)
            
plt.show()

# plt.arrow(jPCA1_traces[target,n_pts-10],jPCA2_traces[target,n_pts-10],
#              dx, dy, color=c, width=0.0005)

In [ ]:
# Find average absolute value for deviations in the preferred directions
sig_indexes = [1,2,4,5,7,8,10,11]
mean_abs_sig_ang_dev = np.mean(np.abs(prf_angs[sig_indexes] - prf_angsB[sig_indexes]))
print("Mean PD deviation for largest PDs (degrees): %f" % (mean_abs_sig_ang_dev*180./np.pi))

mean_abs_ang_dev = np.mean(np.abs(prf_angs - prf_angsB))
print("Mean PD deviation for all PDs (degrees): %f" % (mean_abs_ang_dev*180./np.pi))

---
---
---
---

The cells below are used to produce videos from the simulation.  

In [ ]:
# prepare data for animation
from numpy import cos, sin
if not 'targets' in locals():
    targets = np.array([[0.3, 0.3]])
    times = times_l
    data = data_l
    t_pres = t_pres_l
start_time = 0.
start_idx = int(start_time/net.min_delay)
fdelay = 2000 # number of frames to plot for hand trajectory
# angles of shoulder and elbow
theta_s = arm_activs[start_idx:,0]
theta_e = arm_activs[start_idx:,2]
phi = theta_s + theta_e # elbow angle wrt x axis
# data from tracking units
#acts = np.array(data[1])
ipx = data[ipx_track,start_idx:]
ipy = data[ipy_track,start_idx:]
ten = arm_activs[start_idx:, np.array(range(4,10))].transpose()
# coordinates of hand and elbow
l1 = net.plants[P].l_arm
l2 = net.plants[P].l_farm
xe = cos(theta_s)*l1
ye = sin(theta_s)*l1
xh = xe + cos(phi)*l2
yh = ye + sin(phi)*l2

In [ ]:
#Animation of the arm and muscles
%matplotlib widget
from matplotlib.animation import FuncAnimation
# creating the figure and axis
fig = plt.figure(figsize=(8,8))
plt.axis('equal')
ax = plt.gca()
lim = l1 + l2
ax.set_xlim([-.2, lim])
ax.set_ylim([-.2, lim])
ax.grid()
ax.scatter(targets[:,0], targets[:,1], s=150, c='cyan')
# creating lines and points
line, = ax.plot([], [], 'o-k', lw=2)
points, = ax.plot([], [], '+k')
target, = ax.plot([], [], 'gD')
traj, = ax.plot([], [], 'tab:gray', lw=1)
pred_vel, = ax.plot([], [], 'r-o')
ax.scatter([0.3], [0.3], s=200, c='cyan')
# preparing a colormap for the tensions
ten_max = np.mean(np.max(ten, axis=1))
ten_min = np.mean(np.min(ten, axis=1))
for row_idx, row in enumerate(ten):
    for ent_idx, entry in enumerate(row):
        if entry > 0:
            ten[row_idx, ent_idx] = entry/ten_max
        else:
            ten[row_idx, ent_idx] = entry/abs(ten_min)
#ten = (ten / 2.) + 0.5 # we'll have only positive tensions
mus_lines = []
#cmap=plt.get_cmap('Reds')
#cmap=plt.get_cmap('coolwarm')
cmap=plt.get_cmap('bwr')
for i in range(6):
    mus_lines.append(ax.plot([], [], color=cmap(0.5))[0])
# stuff used to plot the target
#strt_idx = int(np.round(times[0]/t_pres)) # initial index in m_idxs
strt_idx = int(np.round((times[0]+start_time)/t_pres)) # initial index in m_idxs

def init():
    line.set_data([], [])
    points.set_data([], [])
    traj.set_data([], [])
    pred_vel.set_data([], [])
    for i in range(6):
        mus_lines = []
        mus_lines.append(ax.plot([], [], color=cmap(0.5))[0])
    return line, points, mus_lines

def update(frame):
    coord_x = [0, xe[frame], xh[frame]]
    coord_y = [0, ye[frame], yh[frame]]
    ip_x = ipx[:,frame]
    ip_y = ipy[:,frame]
    tens = ten[:, frame]
    line.set_data(coord_x, coord_y)
    points.set_data(ip_x, ip_y)
    #traj.set_data(xh[0:frame], yh[0:frame])
    #traj.set_data(xh[frame-max(0,frame-fdelay):frame], yh[frame-max(0,frame-fdelay):frame])
    if frame > fdelay:
        traj.set_data(xh[frame-fdelay:frame], yh[frame-fdelay:frame])
    for i, ml in enumerate(mus_lines):
        idx = 2*i
        ml.set_data(ip_x[idx:idx+2], ip_y[idx:idx+2])
        ml.set_color(cmap(tens[i]))
    
    cur_time = (frame+start_idx)*net.min_delay
    fig.suptitle('time: ' + '{:f}'.format(cur_time))
    # plotting target
    #cur_idx = int(cur_time/t_pres) + strt_idx
    #cur_idx = int(frame*net.min_delay/t_pres) + strt_idx
    cur_idx = int(frame*net.min_delay/t_pres)  # if using new targets
    x_coord, y_coord = hand_coords[m_idxs[cur_idx]]
    target.set_data([x_coord], [y_coord])
    # plotting predicted velocity
    #pred_vel.set_data([xh[frame], xh[frame]+0.3*pred_v[0][frame]], [yh[frame], yh[frame]+0.3*pred_v[1][frame]])
    #pred_vel.set_data([xh[frame], xh[frame]+5.*pred_v2[0][frame]], [yh[frame], yh[frame]+5.*pred_v2[1][frame]])
    
    return line, points, mus_lines #muscle1

ani = FuncAnimation(fig, update, frames=np.arange(0, len(theta_s), 20), init_func=init, blit=True, interval=20)

#plt.show()

In [ ]:
# alternate way to display the animation
from IPython.display import HTML
#HTML(ani.to_jshtml(fps=20))
HTML(ani.to_html5_video())

---